In [ ]:
import os
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.backend import sigmoid
from sklearn.metrics import confusion_matrix
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import precision_score, make_scorer
from math import sqrt
import threading
from multiprocessing.pool import ThreadPool
import time
import multiprocessing
import traceback
import tensorflow as tf

In [ ]:
import warnings; 
warnings.filterwarnings("ignore")

In [ ]:
from tune_sklearn import TuneGridSearchCV

In [ ]:
def pre_process_data(data,null_threshold):
    """
    Drops Date and Unix Date columns from the data.
    Drops the columns which has null values more than specified null_threshold.
    Replaces infinite values with NAN.
    Drops the rows which has null values.

    Parameters
    ----------
    data : dataframe

    null_threshold : numeric
        numeric value describing the amount of null values that can be present.

    Returns
    -------
    data : dataframe
        an updated dataframe after performing all the opertaions.
    """
    
    data.drop(columns=['Unix Date','Date'],axis=1,inplace=True)
    total = data.shape[0]
    for col in data.columns:
        if null_threshold * total / 100 < data[col].isnull().sum():
            data.drop(columns=[col],axis=1,inplace=True)
    data.replace([np.inf, -np.inf], np.nan, inplace=True)
    data = data.apply(pd.to_numeric,errors='coerce')
    data.dropna(axis=0,inplace=True)
    return data

In [ ]:
def dependent_column(data,column):
    """
    Removes all the Next Day columns.
    Removes all the non Growth Rate Columns (GR)
    add the predictor column to list of columns.

    Parameters
    ----------
    data : dataframe

    column : string
        name of the predictor column 

    Returns
    -------
    data : dataframe
        an updated dataframe after performing all the opertaions.
    column : string
        name of the predictor column
    """
    cols = [col for col in data.columns if "next" not in col.lower() and col.lower().endswith("gr")]
    cols.append(column)
    data = data[cols]
    return (data,column)

In [ ]:
def reshape_data_r(x_train, x_test, y_train, y_test):
    x_train = np.array(x_train)
    x_train = np.reshape(x_train, (x_train.shape[0],1, x_train.shape[1]))
    x_test = np.array(x_test)
    x_test = np.reshape(x_test, (x_test.shape[0],1, x_test.shape[1]))
    y_test = np.array(y_test)
    y_test = np.reshape(y_test, (y_test.shape[0],1,))
    y_train = np.array(y_train)
    y_train = np.reshape(y_train, (y_train.shape[0],1,))
    return (x_train, x_test, y_train, y_test)

In [ ]:
from keras import backend as K
def f1_score(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
def build_lstm(input_shape):
    model = Sequential()
    model.add(LSTM(units=32,return_sequences=True,input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(units=32,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=32))
    model.add(Dropout(0.2))
    model.add(Dense(units = 1,activation = 'sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer="Adam", metrics=f1_score)
    return model

In [ ]:
def best_parameters_rnn(X,Y):
    custom_scorer = make_scorer(metrics.f1_score, greater_is_better = True, pos_label = 1)
    batch_size = [25,32,48,64,100]
    epochs = [25,50,75,100]
    param_grid = dict(epochs=epochs,batch_size=batch_size)
    input_shape = (X.shape[1],X.shape[2])
    model = KerasClassifier(build_fn = build_lstm,input_shape=input_shape,verbose=1)
    grid = TuneGridSearchCV(estimator=model, param_grid=param_grid,scoring=custom_scorer,verbose=1,use_gpu=True)
#     grid = GridSearchCV(estimator=model, param_grid=param_grid,scoring=custom_scorer,verbose=0,n_jobs=-1)
    grid_result = grid.fit(X,Y) 
    epoch = grid_result.best_params_['epochs']
    batch_size = grid_result.best_params_['batch_size']
    return (epoch,batch_size)

In [ ]:
def split_dataset(X,Y,t):
    tr = int(len(X)*t)
    tt = len(X) - tr
    xtr = X[:tr]
    xtt = X[tr:tr+tt]
    ytr = Y[:tr]
    ytt = Y[tr:tr+tt]
    return (xtr,xtt,ytr,ytt)
    

In [ ]:
def create_confusion_matrix(y_pred,y_true):
    
    cm = confusion_matrix(y_true,y_pred)
    accuracy = metrics.accuracy_score(y_true,y_pred)
    precision = metrics.precision_score(y_true,y_pred)
    recall = metrics.recall_score(y_true,y_pred)
    f1_score = metrics.f1_score(y_true,y_pred)
    return {"accuracy":accuracy,"precision":precision,"recall":recall,"f1_score":f1_score,"confusion matrix":cm}

In [ ]:
def error_metrics(y_true, y_pred):
    rmse = sqrt(metrics.mean_squared_error(y_true, y_pred))
    mae = metrics.mean_absolute_error(y_true, y_pred)
    mse = metrics.mean_squared_error(y_true, y_pred)
    return {"root_mean_squared_error":rmse,"mean_absolute_error":mae,"mean_squared_error":mse}

In [ ]:
def reshape_data(x_train, x_test, y_train, y_test,units = 60):
    my_x_train = list()
    my_y_train = list()
    my_x_test = list()
    my_y_test = list()
    for i in range(x_train.shape[0]-units):
        my_x_train.append(x_train.iloc[i:i+units,:])
        my_y_train.append(y_train.iloc[i+units,])
    
    my_x_train = np.array(my_x_train)
    my_x_train = np.reshape(my_x_train,(my_x_train.shape[0],my_x_train.shape[1],my_x_train.shape[2]))
    
    my_y_train = np.array(my_y_train)
    my_y_train = np.reshape(my_y_train,(my_y_train.shape[0],1))
    
    for i in range(x_test.shape[0]-units):
        my_x_test.append(x_test.iloc[i:i+units,:])
        my_y_test.append(y_test.iloc[i+units,])
        
    my_x_test = np.array(my_x_test)
    my_x_test = np.reshape(my_x_test,(my_x_test.shape[0],my_x_test.shape[1],my_x_test.shape[2]))
    
    my_y_test = np.array(my_y_test)
    my_y_test = np.reshape(my_y_test,(my_y_test.shape[0],1))
    
    return (my_x_train, my_x_test, my_y_train, my_y_test)

In [ ]:
def create_rnn_classification(df,column,epochs,batch_size,rate):
    df["Target"] = df[column].apply(lambda x : 1 if x >= rate else 0)
    X = df.drop(columns=["Target",column])
    Y = df["Target"]
#     x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,random_state = 0)
    x_train, x_test, y_train, y_test = split_dataset(X,Y,0.70)
#     x_train, x_test, y_train, y_test = reshape_data_r(x_train, x_test, y_train, y_test)
    x_train, x_test, y_train, y_test = reshape_data(x_train, x_test, y_train, y_test,units = 60)

    input_dim = (x_train.shape[1],x_train.shape[2])

    model = KerasClassifier(build_fn = build_lstm, batch_size=batch_size, epochs=epochs,input_shape=input_dim,verbose=0) 
    history = model.fit(x_train,y_train)

    y_pred = model.predict(x_test)
    y_pred = np.array(y_pred)
    y_pred = np.reshape(y_pred, (y_pred.shape[0],1))

    result = {}
    error = error_metrics(y_test, y_pred)
    confusion = create_confusion_matrix(y_test,y_pred)
    result.update(error)
    result.update(confusion)
    return result

In [ ]:
def rnn_classification(df,column = "Next Day Close Price GR"):
    rate_of_growth = [0.01,0.02,0.03,0.04,0.05]
    solution = list()
    for t in rate_of_growth:
        try:
            print("rate = ",t)
            df["Target"] = df[column].apply(lambda x : 1 if x >= t else 0)
            X = df.drop(columns=["Target",column])
            Y = df["Target"]
    #         x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,random_state = 0)
            x_train, x_test, y_train, y_test = split_dataset(X,Y,0.70)
            x_train, x_test, y_train, y_test = reshape_data(x_train, x_test, y_train, y_test)
            epochs,batch_size = best_parameters_rnn(x_train,y_train)
            result = create_rnn_classification(df,column = "Next Day Close Price GR",epochs = epochs,batch_size = batch_size,rate = t)
            result.update({"epochs":epochs,"batch_size":batch_size,"rate_of_growth":t})
            solution.append(result)
        except:
            traceback.print_exc()
            break
    return solution

In [ ]:
total = pd.read_csv("../input/stockdata/all.csv")
inputpath = "../input/stockdata/Data/Stock"

In [ ]:
%%time
for _,row in total.iterrows():
    try:
        security_id = str(row['security id'])
        print(security_id)
        df = pd.read_csv(os.path.join(inputpath,"gr"+str(security_id)+".csv"))
        df = pre_process_data(df,60)
        df,column = dependent_column(df, column = "Next Day Close Price GR")
        rnn_res = rnn_classification(df,column)
        rnn_df = pd.DataFrame(rnn_res)
        rnn_df.to_csv('rnn_'+str(security_id)+".csv",index=None)
    except Exception as e:
        traceback.print_exc()